# Spark Streaming基础 - DStream API

本笔记本介绍Spark Streaming的基础概念和DStream API的使用。Spark Streaming是Spark的一个扩展，用于处理实时数据流。

## 什么是Spark Streaming？

Spark Streaming是一个可扩展、高吞吐量、容错的流处理系统，它可以处理实时数据流。它将实时输入数据流分成批次，然后由Spark引擎处理，生成最终的结果流。

### 核心概念

- **DStream (Discretized Stream)**：离散化流，是Spark Streaming的基本抽象，表示连续的数据流
- **批处理间隔**：将数据流分割成批次的时间间隔
- **窗口操作**：在滑动时间窗口上进行计算
- **检查点**：用于容错和恢复的机制

## 1. 创建StreamingContext

StreamingContext是Spark Streaming的主要入口点。

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
import time
import threading
import socket
import random

# 创建SparkContext
conf = SparkConf().setAppName("DStream基础")
sc = SparkContext(conf=conf)

# 创建StreamingContext，批处理间隔为2秒
ssc = StreamingContext(sc, 2)

print("StreamingContext创建成功")
print(f"批处理间隔: {ssc._batchDuration} 秒")

## 2. 创建DStream

DStream可以从各种数据源创建，包括TCP套接字、文件系统、Kafka等。

### 2.1 从队列创建DStream（用于测试）

In [ ]:
# 创建一个队列用于模拟数据流
import queue

# 创建RDD队列
rdd_queue = []

# 创建一些示例数据
for i in range(5):
    rdd = sc.parallelize([f"batch_{i}_item_{j}" for j in range(10)])
    rdd_queue.append(rdd)

# 从队列创建DStream
queue_stream = ssc.queueStream(rdd_queue, oneAtATime=True)

print("从队列创建的DStream:")
queue_stream.pprint()

### 2.2 从文本文件创建DStream

In [ ]:
# 创建一个目录用于监控新文件
import os
streaming_dir = "/home/jovyan/data/streaming"
os.makedirs(streaming_dir, exist_ok=True)

# 从文件目录创建DStream
file_stream = ssc.textFileStream(streaming_dir)

print(f"监控目录: {streaming_dir}")
print("文件流DStream创建成功")

### 2.3 创建模拟数据生成器

In [ ]:
# 创建一个简单的数据生成器
def generate_sample_data():
    """生成示例数据"""
    products = ['laptop', 'phone', 'tablet', 'watch', 'headphones']
    regions = ['North', 'South', 'East', 'West']
    
    data = []
    for _ in range(10):
        product = random.choice(products)
        region = random.choice(regions)
        price = random.randint(100, 1000)
        timestamp = int(time.time())
        data.append(f"{timestamp},{product},{region},{price}")
    
    return data

# 测试数据生成器
sample_data = generate_sample_data()
print("示例数据:")
for item in sample_data[:5]:
    print(item)

## 3. DStream转换操作

DStream支持类似RDD的转换操作，如map、filter、reduce等。

In [ ]:
# 重新创建StreamingContext用于演示
if 'ssc' in locals():
    ssc.stop(stopSparkContext=False)

ssc = StreamingContext(sc, 2)

# 创建测试数据队列
test_queue = []
for i in range(3):
    data = generate_sample_data()
    rdd = sc.parallelize(data)
    test_queue.append(rdd)

# 创建DStream
lines = ssc.queueStream(test_queue, oneAtATime=True)

print("原始数据流:")
lines.pprint()

In [ ]:
# map操作：解析CSV数据
def parse_line(line):
    parts = line.split(',')
    if len(parts) == 4:
        return {
            'timestamp': int(parts[0]),
            'product': parts[1],
            'region': parts[2],
            'price': int(parts[3])
        }
    return None

parsed_stream = lines.map(parse_line).filter(lambda x: x is not None)

print("解析后的数据流:")
parsed_stream.pprint()

In [ ]:
# filter操作：过滤高价商品
expensive_items = parsed_stream.filter(lambda x: x['price'] > 500)

print("高价商品流:")
expensive_items.pprint()

In [ ]:
# flatMap操作：展开数据
product_stream = parsed_stream.flatMap(lambda x: [(x['product'], x['price'])])

print("产品价格流:")
product_stream.pprint()

## 4. DStream输出操作

输出操作将DStream的数据输出到外部系统。

In [ ]:
# pprint：打印到控制台
parsed_stream.pprint(10)  # 打印每个批次的前10个元素

# saveAsTextFiles：保存到文件
output_dir = "/home/jovyan/data/output/streaming"
parsed_stream.saveAsTextFiles(output_dir)

# foreachRDD：自定义处理
def process_rdd(rdd):
    if not rdd.isEmpty():
        count = rdd.count()
        print(f"处理了 {count} 条记录")
        
        # 计算平均价格
        if count > 0:
            total_price = rdd.map(lambda x: x['price']).sum()
            avg_price = total_price / count
            print(f"平均价格: {avg_price:.2f}")

parsed_stream.foreachRDD(process_rdd)

## 5. 有状态转换

有状态转换允许您在多个批次之间维护状态。

In [ ]:
# 设置检查点目录（有状态操作需要）
checkpoint_dir = "/home/jovyan/data/checkpoint"
ssc.checkpoint(checkpoint_dir)

# updateStateByKey：维护每个键的状态
def update_function(new_values, running_count):
    if running_count is None:
        running_count = 0
    return sum(new_values, running_count)

# 计算每个产品的累计销售数量
product_counts = product_stream.map(lambda x: (x[0], 1))
running_counts = product_counts.updateStateByKey(update_function)

print("产品累计销售数量:")
running_counts.pprint()

## 6. 窗口操作

窗口操作允许您在滑动时间窗口上应用转换。

In [ ]:
# 窗口操作：在6秒的窗口内，每4秒计算一次
windowed_stream = product_stream.window(6, 4)  # 窗口长度6秒，滑动间隔4秒

print("窗口内的数据:")
windowed_stream.pprint()

# 窗口内的聚合操作
windowed_counts = product_stream.map(lambda x: (x[0], 1)).reduceByKeyAndWindow(
    lambda x, y: x + y,  # 聚合函数
    lambda x, y: x - y,  # 逆聚合函数（可选，用于优化）
    6,  # 窗口长度
    4   # 滑动间隔
)

print("窗口内产品计数:")
windowed_counts.pprint()

## 7. 实际案例：实时销售监控

让我们创建一个实时销售监控系统的示例。

In [ ]:
# 重新创建StreamingContext用于完整示例
if 'ssc' in locals():
    ssc.stop(stopSparkContext=False)

ssc = StreamingContext(sc, 3)  # 3秒批处理间隔
ssc.checkpoint("/home/jovyan/data/checkpoint")

# 创建更多测试数据
sales_queue = []
for i in range(5):
    data = generate_sample_data()
    rdd = sc.parallelize(data)
    sales_queue.append(rdd)

# 创建销售数据流
sales_stream = ssc.queueStream(sales_queue, oneAtATime=True)

# 解析销售数据
parsed_sales = sales_stream.map(parse_line).filter(lambda x: x is not None)

print("实时销售监控系统启动...")

In [ ]:
# 1. 实时销售统计
def calculate_sales_stats(rdd):
    if not rdd.isEmpty():
        sales_data = rdd.collect()
        
        total_sales = sum(item['price'] for item in sales_data)
        total_items = len(sales_data)
        avg_price = total_sales / total_items if total_items > 0 else 0
        
        print(f"=== 批次销售统计 ===")
        print(f"总销售额: ${total_sales}")
        print(f"销售数量: {total_items}")
        print(f"平均价格: ${avg_price:.2f}")
        print("=" * 25)

parsed_sales.foreachRDD(calculate_sales_stats)

In [ ]:
# 2. 按地区统计销售额
region_sales = parsed_sales.map(lambda x: (x['region'], x['price']))
region_totals = region_sales.reduceByKey(lambda x, y: x + y)

print("按地区销售额:")
region_totals.pprint()

# 3. 热门产品排行（窗口操作）
product_sales = parsed_sales.map(lambda x: (x['product'], x['price']))
windowed_product_sales = product_sales.reduceByKeyAndWindow(
    lambda x, y: x + y,
    9,  # 9秒窗口
    3   # 3秒滑动
)

# 获取热门产品
def get_top_products(rdd):
    if not rdd.isEmpty():
        top_products = rdd.takeOrdered(3, key=lambda x: -x[1])
        print("=== 热门产品排行 ===")
        for i, (product, sales) in enumerate(top_products, 1):
            print(f"{i}. {product}: ${sales}")
        print("=" * 25)

windowed_product_sales.foreachRDD(get_top_products)

In [ ]:
# 4. 异常检测：检测异常高价商品
def detect_anomalies(rdd):
    if not rdd.isEmpty():
        sales_data = rdd.collect()
        prices = [item['price'] for item in sales_data]
        
        if len(prices) > 1:
            avg_price = sum(prices) / len(prices)
            threshold = avg_price * 2  # 异常阈值：平均价格的2倍
            
            anomalies = [item for item in sales_data if item['price'] > threshold]
            
            if anomalies:
                print("=== 异常检测 ===")
                print(f"检测到 {len(anomalies)} 个异常高价商品:")
                for item in anomalies:
                    print(f"  {item['product']} - ${item['price']} (阈值: ${threshold:.2f})")
                print("=" * 20)

parsed_sales.foreachRDD(detect_anomalies)

## 8. 启动和停止流处理

配置完所有的转换和输出操作后，需要启动StreamingContext。

In [ ]:
# 启动流处理（注意：这会阻塞执行）
print("启动流处理...")
print("处理5个批次后自动停止")

# 在实际应用中，您可能会使用 ssc.start() 和 ssc.awaitTermination()
# 这里我们使用一个定时器来演示
def stop_streaming():
    time.sleep(15)  # 运行15秒
    ssc.stop(stopSparkContext=False)
    print("流处理已停止")

# 在后台线程中启动停止定时器
stop_thread = threading.Thread(target=stop_streaming)
stop_thread.start()

# 启动流处理
ssc.start()
ssc.awaitTermination()

print("流处理完成")

## 9. 练习

现在，让我们通过一些练习来巩固所学知识。

### 练习1：实时单词计数

创建一个实时单词计数程序，统计文本流中每个单词的出现次数。

In [ ]:
# 创建新的StreamingContext
ssc_word = StreamingContext(sc, 2)
ssc_word.checkpoint("/home/jovyan/data/checkpoint_word")

# 创建文本数据
text_data = [
    "hello world spark streaming",
    "spark is great for big data",
    "streaming data processing with spark",
    "hello spark hello world",
    "big data analytics with spark streaming"
]

text_queue = []
for text in text_data:
    rdd = sc.parallelize([text])
    text_queue.append(rdd)

# 创建文本流
text_stream = ssc_word.queueStream(text_queue, oneAtATime=True)

# 实现单词计数
words = text_stream.flatMap(lambda line: line.split(" "))
word_pairs = words.map(lambda word: (word, 1))
word_counts = word_pairs.reduceByKey(lambda x, y: x + y)

# 维护累计单词计数
def update_word_count(new_values, running_count):
    if running_count is None:
        running_count = 0
    return sum(new_values, running_count)

running_word_counts = word_pairs.updateStateByKey(update_word_count)

print("实时单词计数:")
word_counts.pprint()

print("累计单词计数:")
running_word_counts.pprint()

# 启动并运行一段时间
def stop_word_streaming():
    time.sleep(10)
    ssc_word.stop(stopSparkContext=False)

stop_thread = threading.Thread(target=stop_word_streaming)
stop_thread.start()

ssc_word.start()
ssc_word.awaitTermination()

print("单词计数练习完成")

### 练习2：网络流量监控

模拟网络流量监控，检测异常流量模式。

In [ ]:
# 生成网络流量数据
def generate_network_data():
    ips = ['192.168.1.1', '192.168.1.2', '192.168.1.3', '10.0.0.1', '10.0.0.2']
    actions = ['GET', 'POST', 'PUT', 'DELETE']
    
    data = []
    for _ in range(20):
        ip = random.choice(ips)
        action = random.choice(actions)
        bytes_sent = random.randint(100, 10000)
        timestamp = int(time.time())
        data.append(f"{timestamp},{ip},{action},{bytes_sent}")
    
    return data

# 创建StreamingContext
ssc_network = StreamingContext(sc, 3)
ssc_network.checkpoint("/home/jovyan/data/checkpoint_network")

# 创建网络数据流
network_queue = []
for i in range(4):
    data = generate_network_data()
    rdd = sc.parallelize(data)
    network_queue.append(rdd)

network_stream = ssc_network.queueStream(network_queue, oneAtATime=True)

# 解析网络数据
def parse_network_line(line):
    parts = line.split(',')
    if len(parts) == 4:
        return {
            'timestamp': int(parts[0]),
            'ip': parts[1],
            'action': parts[2],
            'bytes': int(parts[3])
        }
    return None

parsed_network = network_stream.map(parse_network_line).filter(lambda x: x is not None)

# 按IP统计流量
ip_traffic = parsed_network.map(lambda x: (x['ip'], x['bytes']))
ip_totals = ip_traffic.reduceByKeyAndWindow(
    lambda x, y: x + y,
    9,  # 9秒窗口
    3   # 3秒滑动
)

# 检测异常流量
def detect_traffic_anomalies(rdd):
    if not rdd.isEmpty():
        traffic_data = rdd.collect()
        
        if traffic_data:
            # 计算平均流量
            total_bytes = sum(bytes_count for ip, bytes_count in traffic_data)
            avg_bytes = total_bytes / len(traffic_data)
            threshold = avg_bytes * 2  # 异常阈值
            
            print(f"=== 网络流量监控 ===")
            print(f"平均流量: {avg_bytes:.2f} bytes")
            print(f"异常阈值: {threshold:.2f} bytes")
            
            anomalies = [(ip, bytes_count) for ip, bytes_count in traffic_data if bytes_count > threshold]
            
            if anomalies:
                print("检测到异常流量:")
                for ip, bytes_count in anomalies:
                    print(f"  {ip}: {bytes_count} bytes")
            else:
                print("未检测到异常流量")
            print("=" * 25)

ip_totals.foreachRDD(detect_traffic_anomalies)

# 启动网络监控
def stop_network_streaming():
    time.sleep(12)
    ssc_network.stop(stopSparkContext=False)

stop_thread = threading.Thread(target=stop_network_streaming)
stop_thread.start()

ssc_network.start()
ssc_network.awaitTermination()

print("网络流量监控练习完成")

## 10. 总结

在本笔记本中，我们学习了：

1. Spark Streaming的基本概念和DStream抽象
2. 如何创建StreamingContext和DStream
3. DStream的转换操作（map、filter、flatMap等）
4. DStream的输出操作（pprint、saveAsTextFiles、foreachRDD）
5. 有状态转换（updateStateByKey）
6. 窗口操作（window、reduceByKeyAndWindow）
7. 实际案例：实时销售监控系统
8. 如何启动和停止流处理
9. 实践练习：单词计数和网络流量监控

DStream API是Spark Streaming的核心，它提供了处理实时数据流的强大功能。虽然在较新的Spark版本中推荐使用结构化流处理，但理解DStream对于掌握流处理概念仍然很重要。

## 下一步

接下来，我们将学习Spark的结构化流处理（Structured Streaming），它提供了更高级的API和更好的性能。请继续学习 `structured-streaming.ipynb` 笔记本。

In [ ]:
# 清理资源
sc.stop()
print("SparkContext已停止")